In [19]:
import torch

# Humandoid MARL
from Humanoid_MARL import envs
from Humanoid_MARL.envs.base_env import GymWrapper, VectorGymWrapper
from Humanoid_MARL.utils.visual import save_video, save_rgb_image
from Humanoid_MARL.utils.torch_utils import save_models, load_models
from Humanoid_MARL.agent.ppo.train_torch import Agent, eval_unroll, get_agent_actions
from Humanoid_MARL.envs.torch_wrapper import TorchWrapper
from IPython.display import HTML, clear_output
from brax.io import html

In [20]:
config = {
        'num_timesteps': 100_000_000,
        'eval_frequency': 10,
        'episode_length': 1000,
        'unroll_length': 10,
        'num_minibatches': 32,
        'num_update_epochs': 8,
        'discounting': 0.97,
        'learning_rate': 3e-4,
        'entropy_cost': 1e-3,
        'num_envs': 2048,
        'batch_size': 512,
        'env_name': "humanoids",
        'render' : True,
        'device' : 'cuda',
        'model_path' : "../models/20240131_134440_ppo.pt",
        'video_length' : 300,
    }
env = envs.create(
        config['env_name'], batch_size=None, episode_length=config['episode_length'], backend="generalized"
    )
env = GymWrapper(env, get_jax_state=True)
env = TorchWrapper(env, device=config['device'], get_jax_state=True)

# env warmup
obs = env.reset()
action = torch.zeros(env.action_space.shape[0] * env.num_agents).to(config['device'])
env.step(action)
agents = load_models(config['model_path'], Agent, device=config['device'])
jax_states = []
num_steps = 10

for i in range(num_steps):
    print(f"{i} / {num_steps}")
    logits, action = get_agent_actions(agents, obs, env.obs_dims)
    jax_state, observation, reward, done, _ = env.step(Agent.dist_postprocess(action))
    jax_states.append(jax_state)


Models loaded from ../models/20240131_134440_ppo.pt
0 / 10
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10


In [21]:
import jax
from Humanoid_MARL import envs

env = envs.get_environment(env_name=config['env_name'],
                           backend='generalized')
state = jax.jit(env.reset)(rng=jax.random.PRNGKey(seed=0))
print(state)

State(pipeline_state=State(q=Array([-6.7118928e-03,  4.3042656e-04,  1.3961190e+00,  1.0013198e+00,
        3.9071245e-03, -1.5546987e-03,  9.6168444e-03,  4.5053866e-03,
       -4.2285300e-03, -4.7093462e-03,  7.5220130e-03, -5.7970760e-03,
        6.7246128e-03,  8.7262709e-03,  3.4222603e-03,  7.3854830e-03,
        1.2563728e-03, -7.2849891e-03,  7.5726788e-03,  8.9844037e-03,
        9.5206220e-04, -3.7044333e-03, -5.3144479e-03, -7.9694772e-03,
        1.0096626e+00, -8.8023376e-03,  1.4051858e+00,  1.0035782e+00,
       -3.5390616e-03, -6.7494512e-03, -3.3475780e-03,  2.9183002e-03,
       -4.4997786e-03,  6.2011797e-03, -4.5819758e-03,  5.3638192e-03,
       -4.3961313e-04, -4.7533987e-03, -6.9950959e-03, -7.6791905e-03,
        9.9092331e-03, -5.6531737e-03, -4.2719124e-03, -4.8912596e-03,
       -9.2896242e-03, -5.6639933e-03,  8.1148129e-03,  9.9418331e-03],      dtype=float32), qd=Array([ 7.5408854e-03, -6.4459611e-03, -5.1531242e-03, -5.2273632e-03,
       -6.1576534e-03, 

In [22]:
HTML(html.render(env.sys, [jax_states])) 

AttributeError: 'list' object has no attribute 'x'